In [25]:
from pythainlp.tag import pos_tag
from pythainlp.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import codecs
import pandas as pd

In [26]:
data_not = []
def Unique(p):
    text = re.sub("\[(.*?)\]", "", p)
    text = re.sub("\[\/(.*?)\]", "", text)

    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False
# Unique("[TABLE]Note4[/TABLE]มี[FOOD]แซ่บ[/FOOD]เนื้อ1ที่[COMMAND_1]ขึ้นของแล้วนะ[/COMMAND_1]")
# data_not[0]
    

In [27]:
def get_data(fileopen):
    with codecs.open(fileopen, 'r', encoding='utf-8-sig') as f:
        lines = f.read().splitlines()
    return [a for a in lines if Unique(a)]

In [28]:
def toolner_to_tag(text):
    text = text.strip()
    text = re.sub("(\[\/(.*?)\])", "\\1***", text)
    text = re.sub("(\[\w+\])", "***\\1", text)
    text2 = []
    for i in text.split('***'):
        if "[" in i:
            text2.append(i)
        else:
            text2.append("[word]"+i+"[/word]")
    text = "".join(text2)
    return text.replace("[word][/word]", "")

In [29]:
def postag(text):
    listtxt = [i for i in text.split('\n') if i!='']
    # print(listtxt[0])
    list_word = []
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\t'+data.split('\t')[2]+'\n'
        i+=1
    
    return text

In [30]:
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern)
def text2conll2002(text,pos=True):
    text = toolner_to_tag(text)
    print(text)
    text = text.replace("''", '"')
    text = text.replace("’", '"').replace("‘", '"')
    tag = tokenizer.tokenize(text)
    conll2002=""
    l = len(tag)
    for j, (tagopen, text, tagclose) in enumerate(tag):
        word_cut = word_tokenize(text,keep_whitespace=False)
        txt5 = ""
        for i, word in enumerate(word_cut):
            if word in ["''", '"']:
                continue
            if i == 0 and j == 0:
                if tagopen != 'word':
                    txt5 += f"{word}\tB-{tagopen}\tSTART\n"
                else:
                    txt5 += f"{word}\tO\tSTART\n"
            else:
                if tagopen != 'word':
                    if l  == j+1 and i == len(word_cut) - 1:
                        if i == 0:
                            txt5 += f"{word}\tB-{tagopen}\tEND\n"
                        else:
                            txt5 += f"{word}\tI-{tagopen}\tEND\n"
                    else:
                        if i == 0:
                            txt5 += f"{word}\tB-{tagopen}\tNone\n"
                        else:
                            txt5 += f"{word}\tI-{tagopen}\tNone\n"
                else:
                    if l  == j+1 and i == len(word_cut) - 1:
                        txt5 += f"{word}\tO\tEND\n"
                    else:
                        txt5 += f"{word}\tO\tNone\n"
        conll2002 += txt5
    if pos == False:
        return conll2002
    return postag(conll2002)

In [31]:
def alldata_list(lists):
    data_all = []
    for data in lists:
        data_num = []
        try:
            txt = text2conll2002(data,pos=True).split('\n')
            for d in txt:
                tt = d.split('\t')
                if d != "":
                    data_num.append((tt[0], tt[1], tt[2], tt[3]))
            data_all.append(data_num)
        except:
            print(data)
    return data_all
# a = alldata_list(["[TABLE]โต๊ะ4[/TABLE]มี[FOOD]แซ่บ[/FOOD]เนื้อ1ที่[COMMAND_1]ขึ้นของแล้วนะ[/COMMAND_1]"])
# a

In [32]:
file_name = "data_tag"
data_list = get_data(file_name+".txt.")
data_list_file = alldata_list(data_list)

[TABLE]Note4[/TABLE][word]มี[/word][FOOD]แซ่บ[/FOOD][word]เนื้อ1ที่[/word][COMMAND_1]ขึ้นของแล้วนะ[/COMMAND_1]
[FOOD]แซ่บเนื้อ[/FOOD][word]โต๊ะสี[/word][COMMAND_2]เสร็จแล้วนะส่งออกได้[/COMMAND_2]
[FOOD]กระเพราแซ่บเนื้อ[/FOOD][TABLE]โต๊ะ7[/TABLE][COMMAND_1]เตรียมของ[/COMMAND_1][QUESTION]แล้วยัง[/QUESTION]
[FOOD]กระเพราแซ่บเนื้อ[/FOOD][TABLE]โต๊ด7[/TABLE][COMMAND_2]เสร็จแล้ว[/COMMAND_2][QUESTION]เอาไปเสิร์ฟแล้วยัง[/QUESTION]
[word]ไม่สวยนะ[/word][FOOD]กระเพราแซ่บเนื้อ[/FOOD][TABLE]โต๊ด4[/TABLE][word]กับ[/word][FOOD]ข้าวไข่ข้นต้มยำทะเล[/FOOD][word]ที่[/word][COMMAND_1]ขึ้นของแล้วนะ[/COMMAND_1]
[FOOD]รองเท้าแซ่บเนื้อ[/FOOD][TABLE]โต๊ะ4[/TABLE][word]กับ[/word][FOOD]ข้าวไข่ต้มต้มยำทะเล[/FOOD][COMMAND_2]เสร็จแล้วนะเตรียมส่งออกได้[/COMMAND_2]
[FOOD]กระเพราแซ่บเนื้อ[/FOOD][TABLE]โต๊ะ5[/TABLE][word]กับ[/word][FOOD]ข้าวไข่ข้นต้มยำทะเล[/FOOD][word]และ[/word][FOOD]ข้าวไก่ย่างสไปซี่[/FOOD][COMMAND_1]เตรียมของ[/COMMAND_1][QUESTION]ให้แล้วยัง[/QUESTION]
[FOOD]ข้าวไก่ย่างสไปซี่[/FOOD][FOOD]ข้าวกระเพราแ

In [33]:
columns = ["WORD_TOKENIZE", "POS", "TAG", "BOUNDARY"]
df = pd.DataFrame(columns=columns)

for idx, item in enumerate(data_list_file):
    temp_df = pd.DataFrame(item, columns=columns)
    df = pd.concat([df, temp_df], ignore_index=True)

sentence_number = 1
start_idx = df[df['BOUNDARY'] == 'START'].index
end_idx = df[df['BOUNDARY'] == 'END'].index

if not start_idx.empty and not end_idx.empty:
    for start, end in zip(start_idx, end_idx):
        df.loc[start:end, 'SENTENCE'] = f"Sentence: {sentence_number}"
        sentence_number += 1

df.to_csv("data_train.csv", index=False)